In [4]:
import numpy as np
import dill
from decimal import Decimal
data_path=  'D:/DAMP/Research/dilldata/CRNN_train_1.dill'

data = dill.load(open(data_path, 'rb')) #list
# print(len(data))

# for item in data:
#     wav = item['audio'][0]
#     sr = item['audio'][1]
#     wav = torch.tensor(wav)
#     wav = wav[None, :].to(torch.float32)
#     item['w2v_2'] = wav2vec_2(wav,sr)
# data

In [ ]:
import dill
data_path=  'D:/DAMP/Research/dilldata/train_1.dill'

data = dill.load(open(data_path, 'rb')) #list
data[0]

In [ ]:
train_dill = open('D:/DAMP/Research/dilldata/w2v_CRNN_train_1.dill', 'wb') 
dill.dump(data, train_dill)

In [ ]:
tnse_embeddings_wav2vec_2 = []
label = []
label_str = []
import numpy as np


def features(feature_sequence,dim):
# Create some random MFCC shaped features as a sequence of 10 values
   # feature_sequence = np.random.random((10, dim))
    # Get the standard deviation
    stddev_features = np.std(feature_sequence, axis=0)
    # Get the mean
    mean_features = np.mean(feature_sequence, axis=0)
    # Get the average difference of the features
    average_difference_features = np.zeros((dim,))
    for i in range(0, len(feature_sequence) - 2, 2):
        average_difference_features += feature_sequence[i] - feature_sequence[i+1]
    average_difference_features /= (len(feature_sequence) // 2)   
    average_difference_features = np.array(average_difference_features)
    # Concatenate the features to a single feature vector
    concat_features_features = np.hstack((stddev_features, mean_features))
    concat_features_features = np.hstack((concat_features_features, average_difference_features))
    return concat_features_features


# for item in data:
#     rating = item['ratings'][0]
#     if rating >= 0.6 or rating < -0.6:
#         item = item['w2v_2'].numpy()
#         item = np.squeeze(item, axis=0)
#         item = features(item,dim=768)
#         tnse_embeddings_wav2vec_2.append(item)
        
for item in data:
    rating = item['ratings'][0]

    item = item['w2v_2'].numpy()
    item = np.squeeze(item, axis=0)
    item = features(item,dim=768)
    tnse_embeddings_wav2vec_2.append(item) 

count = [0,0,0,0,0]
# for item in data:
#     rating = item['ratings'][0]
#     if rating >= 0.6: 
#         rating = 5
#         rating_str = 'best singings'
#         count[4]+=1
#         label.append(rating)
#         label_str.append(rating_str)
#     elif rating < -0.6: 
#         rating = 1
#         rating_str = 'worst singings'
#         count[0]+=1
#         label.append(rating)
#         label_str.append(rating_str)
for item in data:
    rating = item['ratings'][0]
    if rating >= 0.6: 
        rating = 5
        rating_str = 'best singings'
        count[4]+=1

    elif rating >= 0.2: 
        rating = 4
        count[3]+=1
    elif rating >= -0.2: 
        rating = 3
        count[2]+=1
    elif rating >= -0.6: 
        rating = 2
        count[1]+=1
    elif rating < -0.6: 
        rating = 1
        rating_str = 'worst singings'
        count[0]+=1
    label.append(rating)

count



In [ ]:
print(len(label))
len(tnse_embeddings_wav2vec_2)

In [ ]:

from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler

import pandas
def get_scaled_tsne_embeddings(features, perplexity, iteration):
    embedding = TSNE(n_components=2,
                     perplexity=perplexity,
                     n_iter=iteration).fit_transform(features)
    scaler = MinMaxScaler()
    scaler.fit(embedding)
    return scaler.transform(embedding) 

all_json = dict()
tnse_embeddings_mfccs = []
perplexities = [2, 5, 30, 50, 100,150,250,500]
iterations = [250, 500, 1000, 2000, 5000,8000,10000,15000]

for i,perplexity in enumerate(perplexities):
    for j, iteration in enumerate(iterations):
        tsne_mfccs = get_scaled_tsne_embeddings(tnse_embeddings_wav2vec_2,
                                                perplexity,
                                                iteration)
        tnse_embeddings_mfccs.append(tsne_mfccs)

        
        mfcc_key = 'tsnemfcc{}{}'.format(i, j) 
        wavenet_key = 'tsnewavenet{}{}'.format(i, j) 
    


In [ ]:
for i in range(len(perplexities)):
    for j in range(len(iterations )):
        current_plot = i * 8 + j
        

        Feature_1 = tnse_embeddings_mfccs[current_plot].T[0]
        Feature_2 = tnse_embeddings_mfccs[current_plot].T[1]

        dataframe = {"Feature_1":Feature_1,
                "Feature_2":Feature_2,
                'label':label
                }
        df = pandas.DataFrame(dataframe)
        # colors = {'best singings':'tab:red', 'worst singings':'tab:green'}
        # fig, ax = plt.subplots(figsize=(6, 6))
        # ax.scatter(df['Feature_1'], df['Feature_2'], c=df['label'].map(colors))
        # plt.show()

        df.plot(x="Feature_1", y="Feature_2",xlabel='',c='label', ylabel='',kind='scatter', colormap='viridis')
        

In [ ]:
X_tsne = manifold.TSNE(n_components=2, init='random', random_state=5, verbose=1)

In [ ]:
import os
import torchaudio
import IPython
import requests
import torch
import torchaudio

def wav2vec_2(wav,sr):
    torch.random.manual_seed(123)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    bundle = torchaudio.pipelines.WAV2VEC2_BASE
    model = bundle.get_model().to(device)

    #SPEECH_FILE   = "D:/DAMP/SingEval_DAMPcurated/audio_snippets_1/_cups_pitch_perfect/68307032_39376090.wav"
    #SPEECH_FILE   = "D:/DAMP/SingEval_DAMPcurated/audio_snippets_1/_let_it_go/70366197_41259492.wav"
    bundle = torchaudio.pipelines.WAV2VEC2_BASE
    # print("Sample Rate:", bundle.sample_rate)
    # print(model.__class__)
    waveform, sample_rate =wav ,sr
    waveform = waveform.to(device)

    if sample_rate != bundle.sample_rate:
        waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)
    with torch.inference_mode():
        features, _ = model(waveform)
    return features

In [ ]:
SPEECH_FILE   = "D:/DAMP/SingEval_DAMPcurated/audio_snippets_1/_cups_pitch_perfect/68307032_39376090.wav"



waveform, sample_rate = torchaudio.load(SPEECH_FILE)
print(type(waveform))
print(waveform.shape)
print(type(sample_rate))